In [0]:
%pip install databricks-sdk>=0.57.0

In [0]:
%restart_python

In [0]:
dbutils.widgets.text("instance_name", "")

In [0]:
instance_name = dbutils.widgets.get("instance_name")

In [0]:
import psycopg2

from databricks.sdk import WorkspaceClient
import uuid

w = WorkspaceClient()

instance = w.database.get_database_instance(name=instance_name)
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[instance_name])
user = w.current_user.me().user_name

# Connection parameters
conn = psycopg2.connect(
    host = instance.read_write_dns,
    dbname = "databricks_postgres",
    user = user,
    password = cred.token,
    sslmode = "require"
)

# Execute query
with conn.cursor() as cur:
    cur.execute("SELECT version()")
    version = cur.fetchone()[0]
    print(version)
conn.close()